# Bermudan Swaption Pricing

Value a European Swaption using the Hull-White, Black-Derman-Toy and Black-Karasinski models. I analyse relationships to test the model pricing.

WARNING - PROBLEM WITH OPTION VALUES - NEED TO CHECK

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.finutils import *
from financepy.products.rates import *
from financepy.market.curves import *
from financepy.models.FinModelRatesHW import FinModelRatesHW
from financepy.models.FinModelRatesBK import FinModelRatesBK
from financepy.models.FinModelRatesBDT import FinModelRatesBDT

####################################################################
# FINANCEPY BETA Version 0.191 - This build:  17 Jan 2021 at 18:30 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



## Discount Curve

In [64]:
valuationDate = FinDate(1, 1, 2011)
settlementDate = valuationDate

Set up flat discount curve

In [65]:
rate = 0.0625
liborCurve = FinDiscountCurveFlat(valuationDate, rate, FinFrequencyTypes.SEMI_ANNUAL)

# The Underlying Swap

We begin with the underlying swap.

In [66]:
exerciseDate = settlementDate.addTenor("1Y")
swapMaturityDate = settlementDate.addTenor("4Y")
swapFixedCoupon = 0.060
swapFixedFrequencyType = FinFrequencyTypes.SEMI_ANNUAL
swapFixedDayCountType = FinDayCountTypes.ACT_365F
swapType = FinSwapTypes.PAY 

We can value the forward starting swap.

In [67]:
swap = FinIborSwap(exerciseDate,
                    swapMaturityDate,
                    swapType,
                    swapFixedCoupon,
                    swapFixedFrequencyType,
                    swapFixedDayCountType)

In [68]:
swapValue = swap.value(valuationDate, liborCurve, liborCurve)
print("Swap Value: %9.5f"% swapValue)

Swap Value: 6193.31147


In [69]:
swap

OBJECT TYPE: FinIborSwap
OBJECT TYPE: FinFixedLeg
START DATE: 01-JAN-2012
TERMINATION DATE: 01-JAN-2015
MATURITY DATE: 01-JAN-2015
NOTIONAL: 1000000
PRINCIPAL: 0.0
LEG TYPE: FinSwapTypes.PAY
COUPON: 0.06
FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
DAY COUNT: FinDayCountTypes.ACT_365F
CALENDAR: FinCalendarTypes.WEEKEND
BUS DAY ADJUST: FinBusDayAdjustTypes.FOLLOWING
DATE GEN TYPE: FinDateGenRuleTypes.BACKWARD

OBJECT TYPE: FinFloatLeg
START DATE: 01-JAN-2012
TERMINATION DATE: 01-JAN-2015
MATURITY DATE: 01-JAN-2015
NOTIONAL: 1000000
SWAP TYPE: FinSwapTypes.RECEIVE
SPREAD (BPS): 0.0
FREQUENCY: FinFrequencyTypes.QUARTERLY
DAY COUNT: FinDayCountTypes.THIRTY_E_360
CALENDAR: FinCalendarTypes.WEEKEND
BUS DAY ADJUST: FinBusDayAdjustTypes.FOLLOWING
DATE GEN TYPE: FinDateGenRuleTypes.BACKWARD

# Bermudan Swaption Framework

I now examine FinBermudanSwaption and use the tree models to value it. But first let me price it on the tree models as a European option.

### Start By Only Allowing European Exercise

In [70]:
europeanSwaptionPay = FinIborBermudanSwaption(settlementDate,
                                               exerciseDate,
                                               swapMaturityDate,
                                               FinSwapTypes.PAY,
                                               FinExerciseTypes.EUROPEAN,
                                               swapFixedCoupon,
                                               swapFixedFrequencyType,
                                               swapFixedDayCountType)

In [71]:
europeanSwaptionRec = FinIborBermudanSwaption(settlementDate,
                                               exerciseDate,
                                               swapMaturityDate,
                                               FinSwapTypes.RECEIVE,
                                               FinExerciseTypes.EUROPEAN,
                                               swapFixedCoupon,
                                               swapFixedFrequencyType,
                                               swapFixedDayCountType)

### Black Karasinski Model

In [72]:
blackVol = 0.0000020

In [73]:
sigma = blackVol
a = 0.01
numTimeSteps = 101
model = FinModelRatesBK(sigma, a, numTimeSteps)

In [74]:
valuePay = europeanSwaptionPay.value(valuationDate, liborCurve, model)
valueRec = europeanSwaptionRec.value(valuationDate, liborCurve, model)
valuePayRec = valuePay - valueRec
print("PAY Value: %9.5f"% valuePay)
print("REC Value: %9.5f"% valueRec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 33891.54723
REC Value:   0.00000
PAY - REC: 33891.54723


### Black Derman Toy Model

In [75]:
sigma = blackVol
numTimeSteps = 101
model = FinModelRatesBDT(sigma, numTimeSteps)

In [76]:
valuePay = europeanSwaptionPay.value(valuationDate, liborCurve, model)
valueRec = europeanSwaptionRec.value(valuationDate, liborCurve, model)
valuePayRec = valuePay - valueRec
print("PAY Value: %9.5f"% valuePay)
print("REC Value: %9.5f"% valueRec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 33889.10343
REC Value:   0.00000
PAY - REC: 33889.10343


### Hull White Model

In [77]:
sigma = blackVol * rate
a = 0.01
numTimeSteps = 101
model = FinModelRatesHW(sigma, a, numTimeSteps)

In [78]:
valuePay = europeanSwaptionPay.value(valuationDate, liborCurve, model)
valueRec = europeanSwaptionRec.value(valuationDate, liborCurve, model)
valuePayRec = valuePay - valueRec
print("PAY Value: %9.5f"% valuePay)
print("REC Value: %9.5f"% valueRec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 33891.54723
REC Value:   0.00000
PAY - REC: 33891.54723


# Now allow Multiple Exercise Dates - Bermudan

Now allow exercise on coupon dates after expiry.

In [79]:
bermudanSwaptionPay = FinIborBermudanSwaption(settlementDate,
                                               exerciseDate,
                                               swapMaturityDate,
                                               FinSwapTypes.PAY,
                                               FinExerciseTypes.BERMUDAN,
                                               swapFixedCoupon,
                                               swapFixedFrequencyType,
                                               swapFixedDayCountType)

In [80]:
bermudanSwaptionRec = FinIborBermudanSwaption(settlementDate,
                                               exerciseDate,
                                               swapMaturityDate,
                                               FinSwapTypes.RECEIVE,
                                               FinExerciseTypes.BERMUDAN,
                                               swapFixedCoupon,
                                               swapFixedFrequencyType,
                                               swapFixedDayCountType)

### Black Karasinski Model

In [81]:
sigma = 0.20
a = 0.01
numTimeSteps = 201
model = FinModelRatesBK(sigma, a, numTimeSteps)

In [82]:
valuePay = bermudanSwaptionPay.value(valuationDate, liborCurve, model)
valueRec = bermudanSwaptionRec.value(valuationDate, liborCurve, model)
valuePayRec = valuePay - valueRec
print("PAY Value: %9.5f"% valuePay)
print("REC Value: %9.5f"% valueRec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 35716.19440
REC Value: 10240.81073
PAY - REC: 25475.38367


### Black-Derman-Toy Model

In [83]:
sigma = 0.20
numTimeSteps = 201
model = FinModelRatesBDT(sigma, numTimeSteps)

In [84]:
valuePay = bermudanSwaptionPay.value(settlementDate, liborCurve, model)
valueRec = bermudanSwaptionRec.value(settlementDate, liborCurve, model)
valuePayRec = valuePay - valueRec
print("PAY Value: %9.5f"% valuePay)
print("REC Value: %9.5f"% valueRec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 35879.78862
REC Value: 10463.07468
PAY - REC: 25416.71394


### Hull-White Model

In [85]:
sigma = 0.01
a = 0.01
numTimeSteps = 201
model = FinModelRatesHW(sigma, a)

In [86]:
valuePay = bermudanSwaptionPay.value(settlementDate, liborCurve, model)
valueRec = bermudanSwaptionRec.value(settlementDate, liborCurve, model)
valuePayRec = valuePay - valueRec
print("PAY Value: %9.5f"% valuePay)
print("REC Value: %9.5f"% valueRec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 35253.56980
REC Value: 8298.90257
PAY - REC: 26954.66724


The swaption is Bermudan so it can only be exercised on coupon payment dates

Copyright (c) 2020 Dominic O'Kane